## Notebook to process GNSS data for each receiver

### Step 1: Load libraries

Chunk that sloads necessary packages & sets working environment to where the jupyterlab notebook file is 

In [2]:
%load_ext autoreload
%autoreload 2
import gnssvod as gv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pdb
import importlib
import zipfile
import os
import xarray as xr
import glob
import shutil
from gnssvod.hemistats.hemistats import hemibuild
#import georinex as gr
#import qgrid as interactive table 
from matplotlib.collections import PatchCollection
import matplotlib.dates as mdates
import shutil
from matplotlib import rcParams
from datetime import datetime, timedelta

### Step 2: Time intervals

Set up time intervals for the study period. The argument **periods** represents the number of days from the start day.

In [3]:
startday = pd.to_datetime('01-03-2025',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

IntervalIndex([[2025-03-01 00:00:00, 2025-03-02 00:00:00),
               [2025-03-02 00:00:00, 2025-03-03 00:00:00),
               [2025-03-03 00:00:00, 2025-03-04 00:00:00),
               [2025-03-04 00:00:00, 2025-03-05 00:00:00),
               [2025-03-05 00:00:00, 2025-03-06 00:00:00),
               [2025-03-06 00:00:00, 2025-03-07 00:00:00),
               [2025-03-07 00:00:00, 2025-03-08 00:00:00),
               [2025-03-08 00:00:00, 2025-03-09 00:00:00),
               [2025-03-09 00:00:00, 2025-03-10 00:00:00),
               [2025-03-10 00:00:00, 2025-03-11 00:00:00)],
              dtype='interval[datetime64[ns], left]')

### Step 3: Read RINEX files

We then proceed to read the RINEX files. This recquires to indicate the location of the files (**pattern**) and the location of the output directory (**outputdir**). Sometimes we could get the following error:

ValueError: Missing an approximate antenna position. Provide the argument ‘approx_position’ to preprocess()

Make sure to update to your own path.

In [3]:
# option for batch processing, example with my files- again, approx-position is for an error
#batch processing for one station
pattern= {'YoungPine-2':'/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_*.25O'}
outputdir={'YoungPine-2':'/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/youngpine_pheno_nc/'  }
#approx_position=[2481170.8276 -5525721.2902,1992311.7021 ]
keepvars=['S1C','S1X','S2C','S2X']
#gv.preprocess(pattern,interval='30s',keepvars=keepvars,outputdir=outputdir,approx_position=approx_position)
gv.preprocess(pattern,interval='30s',keepvars=keepvars,outputdir=outputdir)

Created a temporary directory at /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn
Reach_raw_20250311193931.nc already exists, skipping.. (pass overwrite=True to overwrite)
/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250328193951.25O exists | Reading...
Observation file  /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250328193951.25O  is read in 37.46 seconds.
Processing 3043179 individual observations
Calculating Azimuth and Elevation
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250870000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2359//GFZ0MGXRAP_20250870000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250870000_01D_05M_ORB.SP3.gz: 1.05MB [00:02, 385kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2359//GFZ0MGXRAP_20250870000_01D_05M_ORB.SP3.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250870000_01D_05M_ORB.SP3 file is read in 3.20 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250880000_01D_05M_ORB.SP3
Downloading: ftp://gssc.esa.int/gnss/products/2359//GFZ0MGXRAP_20250880000_01D_05M_ORB.SP3.gz

GFZ0MGXRAP_20250880000_01D_05M_ORB.SP3.gz: 1.05MB [00:02, 389kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2359//GFZ0MGXRAP_20250880000_01D_05M_ORB.SP3.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250880000_01D_05M_ORB.SP3 file is read in 3.09 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250870000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250870000_01D_30S_CLK.CLK.gz

...

 | Download completed for GFZ0MGXRAP_20250800000_01D_30S_CLK.CLK.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250800000_01D_30S_CLK.CLK file is read in 1.16 seconds
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250810000_01D_30S_CLK.CLK exists | Reading...
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250810000_01D_30S_CLK.CLK file is read in 1.09 seconds
SP3 interpolation is done in 0.89 seconds
Saved 103325 individual observations in /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/youngpine_pheno_nc/Reach_raw_20250321193943.nc
/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250327193950.25O exists | Reading...
Observation file  /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250327193950.25O  is read in 37.83 seconds.
Processing 307

GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3.gz: 1.04MB [00:02, 392kB/s]                             


 | Download completed for ftp://gssc.esa.int/gnss/products/2357//GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250710000_01D_05M_ORB.SP3 file is read in 3.08 seconds
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250720000_01D_05M_ORB.SP3 exists | Reading...
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250720000_01D_05M_ORB.SP3 file is read in 0.20 seconds
This file does not exist: /var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK
Downloading: GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz

GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz: 4.82MB [00:04, 1.06MB/s]                            


 | Download completed for GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK.gz
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250710000_01D_30S_CLK.CLK file is read in 1.17 seconds
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250720000_01D_30S_CLK.CLK exists | Reading...
/var/folders/zy/x63zv75j4zxbjxjc77btggbr0000gt/T/tmpyo30vvfn/GFZ0MGXRAP_20250720000_01D_30S_CLK.CLK file is read in 1.13 seconds
SP3 interpolation is done in 0.82 seconds
Saved 102865 individual observations in /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/youngpine_pheno_nc/Reach_raw_20250312193932.nc
/Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250314193935.25O exists | Reading...
Observation file  /Users/coopercollins/Box/Project_MetoliusGNSS/VOD/Data/GNSS/extracted/youngpine/youngpine_pheno/Reach_raw_20250314193935.25O  is read in 37.46 seconds.
Processing 306

Continue portion in a separate file, we call ours gnss_gathering

In [ ]:
#pairing and concatenating--- starting by establishing path to .nc files per station, replace with own path 
pattern={'MACROCOSM-1':'/Volumes/coraldelmar/march_collection/MACROCOSM-1/nc/*.nc',
        'MACROCOSM-2':'/Volumes/coraldelmar/march_collection/MACROCOSM-2/nc/*.nc',
        'MACROCOSM-3':'/Volumes/coraldelmar/march_collection/MACROCOSM-3/nc/*.nc',
        'MACROCOSM-5':'/Volumes/coraldelmar/march_collection/MACROCOSM-5/nc/*.nc',
        'MACROCOSM-7':'/Volumes/coraldelmar/march_collection/MACROCOSM-7/nc/*.nc'}

In [ ]:
#re-establish time intervals - this will be necessary when you want to look at a specific period of data. 
startday = pd.to_datetime('01-03-2025',format='%d-%m-%Y')
timeintervals=pd.interval_range(start=startday, periods=10, freq='D', closed='left')
timeintervals

In [ ]:
#concatenating. Reference station always goes first. Replace with own path+pairings


#MACROCOSM 1.5
pairings={'MACROCOSM-1.5':('MACROCOSM-1','MACROCOSM-5')}
outputdir={'MACROCOSM-1.5':'/Volumes/coraldelmar/march_collection/MACROCOSM-1.5'}
keepvars=['S*','Azimuth','Elevation']
out=gv.gather_stations(pattern,pairings,timeintervals,outputresult=True)

The portion below I adapt for different station pairings

In [ ]:
#
ds=xr.open_mfdataset('/Volumes/coraldelmar/march_collection/MACROCOSM-5.7/*.nc',combine='nested',concat_dim='Epoch')
ds

In [ ]:
#
df=ds.to_dataframe().dropna(how='all').reorder_levels(["Station","Epoch","SV"]).sort_index()
df

In [ ]:
hemi = gv.hemibuild(4)
hemi

In [ ]:
hemi.grid

In [ ]:
# get patches
patches = hemi.patches() 
patches

In [ ]:
newdf = hemi.add_CellID(df)
newdf

In [ ]:
hemi_average = newdf.groupby(['CellID','Station']).mean()
hemi_average

In [ ]:
# set which data files should be loaded
pattern = '/Volumes/coraldelmar/march_collection/MACROCOSM-5.7/*.nc'
# define how to associate stations together. Always put reference station first.
pairings={'MACROCOSM-5.7':('MACROCOSM-5','MACROCOSM-7')}
# define if some observables with similar frequencies should be combined together. In the future, this should be replaced by the selection of frequency bands.
bands = {'VOD1':['S1','S1X','S1C'],'VOD2':['S2','S2X','S2C']}

vod = gv.calc_vod(pattern,pairings,bands)
vod = vod['MACROCOSM-5.7']
vod

In [ ]:
# intialize hemispheric grid
hemi = gv.hemibuild(2)
# get patches for plotting later
patches = hemi.patches()
# classify vod into grid cells, drop azimuth and elevation afterwards as we don't need it anymore
vod = hemi.add_CellID(vod)
# get average value per grid cell
vod_avg = vod.groupby(['CellID']).agg(['mean', 'std', 'count'])
# flatten the columns
vod_avg.columns = ["_".join(x) for x in vod_avg.columns.to_flat_index()]
vod_avg

In [ ]:
# merge statistics with the original VOD measurements
vod_anom = vod.join(vod_avg,on='CellID')

In [ ]:
vod_anom['VOD1_anom'] = vod_anom['VOD1']-vod_anom['VOD1_mean']
vod_anom['VOD2_anom'] = vod_anom['VOD2']-vod_anom['VOD2_mean']

In [ ]:
vod_ts = vod_anom.groupby(pd.Grouper(freq='1h', level='Epoch')).mean()
vod_ts

In [ ]:
#i export vod_anom because it is what we use for plotting moving forward.
vod_anom=vod_anom.reset_index()
vod_anom.to_csv('/Volumes/coraldelmar/march_collection/MACROCOSM-5.7/vod_anom.csv', index=False)